## Basic methods plus correlation pipeline

Now we use a combination of constant, quasi-constant, and correlation methods. Finally, we will compare the performance of ML models built with the different feature subsets.

### Putting it all together

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

**load the dataset!**

In [2]:
data = pd.read_csv('../dataset_1.csv')
data.shape

(50000, 301)

**Separate dataset into train and test!**

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['target'], axis=1),
    data['target'],
    test_size=0.3,
    random_state=0)
X_train.shape, X_test.shape

((35000, 300), (15000, 300))

**Copy of the dataset to measure the ML models performance!**

In [4]:
X_train_original = X_train.copy()
X_test_original = X_test.copy()

**1. Remove constant features!**

In [5]:
constant_features = [feat for feat in X_train.columns if X_train[feat].std() == 0]
X_train.drop(labels=constant_features, axis=1, inplace=True)
X_test.drop(labels=constant_features, axis=1, inplace=True)
X_train.shape, X_test.shape

((35000, 266), (15000, 266))

**2. Remove quasi-constant features!**

In [6]:
sel = VarianceThreshold(threshold=0.01)  # 0.1 means 99% of observations!
sel.fit(X_train)  # fit finds the features with low variance
sum(sel.get_support()) # how many not quasi-constant?

215

In [7]:
features_to_keep = X_train.columns[sel.get_support()]

**Then remove the features like this!**

In [8]:
X_train = sel.transform(X_train)
X_test = sel.transform(X_test)
X_train.shape, X_test.shape

((35000, 215), (15000, 215))

**Sklearn transformations are numpy arrays! Chane them to Data Frames!**

In [9]:
X_train= pd.DataFrame(X_train)
X_train.columns = features_to_keep
X_test= pd.DataFrame(X_test)
X_test.columns = features_to_keep

**3. Remove duplicated features!**

In [10]:
duplicated_feat = []
for i in range(0, len(X_train.columns)):
    if i % 10 == 0: 
        print(i)
    col_1 = X_train.columns[i]
    for col_2 in X_train.columns[i + 1:]:
        if X_train[col_1].equals(X_train[col_2]):
            duplicated_feat.append(col_2)
len(duplicated_feat)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210


10

**Remove duplicated features!**

In [11]:
X_train.drop(labels=duplicated_feat, axis=1, inplace=True)
X_test.drop(labels=duplicated_feat, axis=1, inplace=True)
X_train.shape, X_test.shape

((35000, 205), (15000, 205))

**Keep a copy of the dataset except constant and duplicated variables!**

In [12]:
X_train_basic_filter = X_train.copy()
X_test_basic_filter = X_test.copy()

**Find and remove correlated features!**

In [13]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr
corr_features = correlation(X_train, 0.8)
print('correlated features: ', len(set(corr_features)) )

correlated features:  93


In [14]:
X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)
X_train.shape, X_test.shape

((35000, 112), (15000, 112))

### Compare performance in machine learning models

**This function builds random forests and compare performance in train and test set!**

In [15]:
def run_randomForests(X_train, X_test, y_train, y_test):
    rf = RandomForestClassifier(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)
    print('Train set')
    pred = rf.predict_proba(X_train)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
    print('Test set')
    pred = rf.predict_proba(X_test)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

**Original dataset (all variables) !**

In [16]:
run_randomForests(X_train_original,
                  X_test_original,
                  y_train, y_test)

Train set
Random Forests roc-auc: 0.807612232524249
Test set
Random Forests roc-auc: 0.7868832427636059


**Filter methods - basic**

In [17]:
run_randomForests(X_train_basic_filter,
                  X_test_basic_filter,
                  y_train, y_test)

Train set
Random Forests roc-auc: 0.810290026780428
Test set
Random Forests roc-auc: 0.7914020645941601


**Filter methods - correlation !**

In [18]:
run_randomForests(X_train,
                  X_test,
                  y_train, y_test)

Train set
Random Forests roc-auc: 0.8066004772684517
Test set
Random Forests roc-auc: 0.7859521124929707


**We removed constant, quasi-constant, duplicated and correlated features, the feature space  reduced dramatically (from 300 to 112), without affecting the performance of the random forests (0.786 vs 0.7859).**

**This logistic regression function compares performance in train and test set**

In [19]:
def run_logistic(X_train, X_test, y_train, y_test):
    logit = LogisticRegression(random_state=44, max_iter=500)
    logit.fit(X_train, y_train)
    print('Train set')
    pred = logit.predict_proba(X_train)
    print('Logistic Regression roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
    print('Test set')
    pred = logit.predict_proba(X_test)
    print('Logistic Regression roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

**For logistic regression features need to be in the same scale!**

In [20]:
scaler = StandardScaler().fit(X_train_original)
run_logistic(scaler.transform(X_train_original),
             scaler.transform(X_test_original), y_train, y_test)

Train set
Logistic Regression roc-auc: 0.8028294003037697
Test set
Logistic Regression roc-auc: 0.7951005875022253


**Filter methods - basic !**

In [21]:
scaler = StandardScaler().fit(X_train_basic_filter)
run_logistic(scaler.transform(X_train_basic_filter),
             scaler.transform(X_test_basic_filter),
                  y_train, y_test)

Train set
Logistic Regression roc-auc: 0.8022720222681473
Test set
Logistic Regression roc-auc: 0.7947452037606925


**Filter methods - correlation !**

In [22]:
scaler = StandardScaler().fit(X_train)
run_logistic(scaler.transform(X_train),
             scaler.transform(X_test),
                  y_train, y_test)

Train set
Logistic Regression roc-auc: 0.7942736215203425
Test set
Logistic Regression roc-auc: 0.7881862599647895


Similarly, for logistic regression, removing constant, quasi-constant, duplicated and highly correlated features, did not affect dramatically the performance of the algorithm.